In [856]:
import pandas as pd
from pandas import DataFrame as df
import scipy.signal as signal
import matplotlib.pyplot as plt

import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [874]:
class session():
    # An object for storing data and metadata for one session of EEG recording
    def __init__(self, filename):
        self.f = filename
        
    def get_data(self, type='OpenBCI', fix=['OpenBCI-col_names']):
        # Create a pandas dataframe with channel data (data_chns)
        # fix:
        #   - "OpenBCI-col_names" - removes spaces from column names and makes channel number's 1-indexed
        if type == 'OpenBCI':
            f = open(self.f)
            self.meta = [f.readline() for i in range(4)]
            self.n_chns = int(self.meta[1][22:])
            self.fs = int(self.meta[2][15:18]) # sample rate

            self.data_chns = pd.read_csv(self.f, skiprows=[0,1,2,3])
            self.data_chns = self.data_chns.drop(columns=['Sample Index'])
            
            if 'OpenBCI-col_names' in fix:
                for i in range(self.n_chns):
                    self.data_chns = self.data_chns.rename(columns={self.data_chns.keys()[i]:'eeg_channel_'+str(i+1)})
            
            if 'Time' not in self.data_chns.keys():
                self.data_chns.reset_index(inplace=True)
                self.data_chns = self.data_chns.rename(columns={'index':'Time'})
                self.data_chns['Time'] = self.data_chns['Time'].divide(self.fs)
                    
    def crop_data(self, upto=-1, after=-1):
        # Crop the data upto or after a certain time in seconds
        if upto > 0:
            self.data_chns = self.data_chns.query('Time>=@upto')
            print('Removed data upto ' + str(upto) + ' seconds.\n')
        if after > 0:
            self.data_chns = self.data_chns.query('Time<=@after')
            print('Removed data after ' + str(upto) + ' seconds.\n')
    
    def plot(self, ver='', chns=[]):
        # Plot the data in different helpful ways
        if ver == 'pick-channels':
            fig = go.Figure()
            
            if len(chns) == 0:
                chns = range(1, self.n_chns + 1)
                print(chns)
            for i in range(len(chns)):
                fig.add_trace(go.Scatter(x=self.data_chns['Time'],
                                            y=self.data_chns['eeg_channel_'+str(chns[i])],
                                            mode='lines',
                                            name='EEG Ch.'+str(chns[i])))
            fig.show()

        if ver == 'all-chns-in-one':
            fig = go.Figure()
            for i in range(self.n_chns):
                fig.add_trace(go.Scatter(x=self.data_chns['Time'], y=self.data_chns['eeg_channel_'+str(i+1)],
                        mode='lines',
                        name='EEG Ch.'+str(i+1)))
            fig.show()
        if ver == 'chn-grid':
            fig = make_subplots(rows=self.n_chns, cols=1, shared_xaxes=True,
                                vertical_spacing=0.01,
                                subplot_titles=['EEG Channel '+str(i+1) for i in range(self.n_chns)])
            for i in range(self.n_chns):
                fig.append_trace(go.Scatter(x=self.data_chns['Time'],
                                            y=self.data_chns['eeg_channel_'+str(i+1)],
                                            mode='lines',
                                            name='EEG Ch.'+str(i+1)),
                                            row=i+1,
                                            col=1)
            fig.update_layout(height=300*self.n_chns, width=800, title_text="Data by Channel")
            fig.show()
            
    def preprocess(self, hpf=0, lpf=0, inplace=False):
        # Handle preprocessing to create a dataframe of processed data
        # High-pass filter = hpf
        # Low-pass filter = lpf
        
        self.proc_data_chns = pd.DataFrame()
        
        if lpf > 0:
            for i in range(self.n_chns):
                b, a = signal.butter(2, lpf, 'low', fs=self.fs)
                self.proc_data_chns['eeg_channel_'+str(i+1)] = signal.filtfilt(b, a, self.data_chns['eeg_channel_'+str(i+1)])
        self.proc_data_chns['Time'] = self.data_chns['Time']
        if inplace:
            self.data_chns = self.proc_data_chns
        return self.proc_data_chns

In [875]:
S1 = session('OpenBCI-RAW-2021-08-13_18-03-39.txt')
S1.get_data()
S1.crop_data(upto=1)

Removed data upto 1 seconds.



In [876]:
S1.preprocess(lpf=30, inplace=True)

,eeg_channel_1,eeg_channel_2,eeg_channel_3,eeg_channel_4,eeg_channel_5,eeg_channel_6,eeg_channel_7,eeg_channel_8,eeg_channel_9,eeg_channel_10,eeg_channel_11,eeg_channel_12,eeg_channel_13,eeg_channel_14,eeg_channel_15,eeg_channel_16,Time
0,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.131714e+06,-2.267673e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,NaN
1,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.129599e+06,-2.264694e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,NaN
2,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.129007e+06,-2.263922e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,NaN
3,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.129114e+06,-2.264246e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,NaN
4,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.129107e+06,-2.264367e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1135,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.139446e+06,-2.275200e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,9.080
1136,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.139535e+06,-2.275325e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,9.088
1137,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.139783e+06,-2.275611e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,9.096
1138,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.139485e+06,-2.275156e+06,-4500000.375,-4500000.375,-4500000.375,4500000.0,4500000.0,-4500000.375,-4500000.375,-4500000.375,9.104


In [877]:
# plot
S1.plot(ver='chn-grid', chns=[7,8])

In [818]:
S1.data_chns

,Time,eeg_channel_1,eeg_channel_2,eeg_channel_3,eeg_channel_4,eeg_channel_5,eeg_channel_6,eeg_channel_7,eeg_channel_8,eeg_channel_9,...,Other.2,Other.3,Other.4,Other.5,Other.6,Analog Channel 0,Analog Channel 1,Analog Channel 2,Timestamp,Timestamp (Formatted)
125,1.000,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.131713e+06,-2.267672e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:41.192
126,1.008,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.126519e+06,-2.260492e+06,-4500000.375,...,96.0,14.0,112.0,6.0,104.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:41.192
127,1.016,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.133193e+06,-2.269660e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:41.192
128,1.024,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.124751e+06,-2.258046e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:41.192
129,1.032,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.133782e+06,-2.271300e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:41.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1260,10.080,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.136316e+06,-2.270576e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:50.185
1261,10.088,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.141993e+06,-2.279065e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:50.185
1262,10.096,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.137582e+06,-2.272248e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:50.185
1263,10.104,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-4500000.375,-2.141166e+06,-2.277670e+06,-4500000.375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.628899e+09,2021-08-13 18:03:50.185
